# Google Trends Data using [pytrends](https://github.com/GeneralMills/pytrends)

---

In [1]:
# === General imports === #
import pandas as pd
import time

In [2]:
# === Connect to Google === #
from pytrends.request import TrendReq

# Mountain Time offset is -7:00 = 420 minutes
pytrends = TrendReq(hl="en-US", tz=420)

---

### Automate!

The goal here is to...

- Iterate through the full list of keywords
- Send off 5 at a time to Google Trends
- Concatenate the resulting data into a single dataframe

In [4]:
# === Import items into list === #

# Open text file and read, splitting on the newline character
# For this version, all the # plastics are grouped into their parent
# e.g. #1 Plastic, #2 Plastic, etc.
with open("earth911_items_list_plastic.txt", "r") as f:
    recycle_list = f.read().splitlines()

# Confirm it worked as expected
# assert len(recycle_list) == 332
len(recycle_list)  # This version is different (294 items)

294

In [5]:
# === Preview the resulting list === #
recycle_list[:20]

['#1 Plastic',
 '#2 Plastic',
 '#3 Plastic',
 '#4 Plastic',
 '#5 Plastic',
 '#6 Plastic',
 '#7 Plastic',
 'Acids',
 'Adhesives',
 'Adult Toys',
 'Aerosol Cans - Full',
 'Air Conditioners',
 'Alkaline Batteries',
 'Aluminum Beverage Cans',
 'Aluminum Foil',
 'Aluminum Food Cans',
 'Aluminum Pie Plates',
 'Aluminum Trays',
 'Ammunition',
 'Antifreeze']

In [7]:
# === Create OG Dataframe === #
df_blank = pd.read_csv("blank_date_index.csv", index_col="date")
df_blank

""
date
2015-01-25
2015-02-01
2015-02-08
2015-02-15
2015-02-22
...
2019-12-22
2019-12-29
2020-01-05


In [24]:
def build_trend_df(base_query: str, kw_list: list, df) -> None:
    """
    Google Trends for 5 keywords at a time and add to dataframe.
    
    Parameters
    ----------
    base_query : string
        Query onto which each keyword is appended.
    kw_list : list
        List of keywords.
    df : pd.DataFrame
        Pre-existing dataframe onto which trends data will be added.
    
    Returns
    -------
    None
        Trends data is added to pre-existing dataframe, not returned.
    """

    s_prev = 0  # Used to keep track of previous loop value
    # Build lists of 5 items at a time
    for s in range(5, 296, 5):  # Updated with proper length of input list
        # TODO: make list length an parameter
        five_items_list = kw_list[s_prev:s]  # Slice from prev value of s to current

        # Concatenate each keyword with the base word - returns new list
        concat_queries = [" ".join([base_query, kw]) for kw in five_items_list]

        # Build the payload and run the query for current 5 items
        pytrends.build_payload(concat_queries)
        df_return = pytrends.interest_over_time()

        # Add any current results to OG dataframe
        if df_return.empty:
            print(f"No results for {s_prev}-{s}.")
            print(concat_queries)
        else:
            for item in concat_queries:
                if item in df_return.columns:
                    df[item] = df_return[item]

        # Space out the requests and give visual progress status
        print(f"Completed items {s_prev}-{s}.")
        time.sleep(2)
        s_prev = s  # Set current value of s to use in next iteration

---

### `"How to recycle" + {keyword}`

In [10]:
# === Run the function! === #
base_q = "how to recycle"

build_trend_df(base_q, recycle_list, df_blank)

No results for 0-5.
['how to recycle #1 Plastic', 'how to recycle #2 Plastic', 'how to recycle #3 Plastic', 'how to recycle #4 Plastic', 'how to recycle #5 Plastic']
Completed items 0-5.
No results for 5-10.
['how to recycle #6 Plastic', 'how to recycle #7 Plastic', 'how to recycle Acids', 'how to recycle Adhesives', 'how to recycle Adult Toys']
Completed items 5-10.
Completed items 10-15.
No results for 15-20.
['how to recycle Aluminum Food Cans', 'how to recycle Aluminum Pie Plates', 'how to recycle Aluminum Trays', 'how to recycle Ammunition', 'how to recycle Antifreeze']
Completed items 15-20.
No results for 20-25.
['how to recycle Arts and Crafts Supplies', 'how to recycle Asbestos', 'how to recycle Asphalt', 'how to recycle Audio Equipment', 'how to recycle Auto Bodies']
Completed items 20-25.
No results for 25-30.
['how to recycle Auto Parts', 'how to recycle Baby Products', 'how to recycle Barbeque Grills', 'how to recycle Bicycles', 'how to recycle Bike Tires']
Completed items

In [12]:
# === Check out the results === #
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
df_blank.describe().T

,count,mean,std,min,25%,50%,75%,max
how to recycle Aerosol Cans - Full,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
how to recycle Air Conditioners,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
how to recycle Alkaline Batteries,261.0,22.417625,25.295749,0.0,0.0,0.0,46.0,100.0
how to recycle Aluminum Beverage Cans,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
how to recycle Aluminum Foil,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
how to recycle Button Cell Batteries,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
how to recycle CD Cases,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
how to recycle CD Players,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
how to recycle CDs,261.0,30.831418,16.147944,0.0,22.0,26.0,39.0,100.0
how to recycle CFLs,261.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [13]:
# === Export to csv for posterity === #
df_blank.to_csv("howtorecycle-pytrends_all.csv")

In [14]:
# === New df to filter out 0-value columns === #
df2 = df_blank.copy()

In [15]:
# === Get list of 0-value columns === #
zero_value_cols = []

for col in df2.columns:
    if df2[col].mean() == 0.0:
        zero_value_cols.append(col)

In [16]:
# === Drop 0-value columns === #
df3 = df2.drop(columns=zero_value_cols)
df3.shape

(261, 32)

In [17]:
# === Result is 78 columns - Check em out === #
df3.describe().T

,count,mean,std,min,25%,50%,75%,max
how to recycle Alkaline Batteries,261.0,22.417625,25.295749,0.0,0.0,0.0,46.0,100.0
how to recycle CDs,261.0,30.831418,16.147944,0.0,22.0,26.0,39.0,100.0
how to recycle Cables,261.0,13.363985,18.598224,0.0,0.0,0.0,30.0,100.0
how to recycle Car Batteries,261.0,19.061303,19.083724,0.0,0.0,29.0,33.0,89.0
how to recycle Carpet,261.0,19.862069,20.759524,0.0,0.0,28.0,33.0,100.0
how to recycle Cell Phones,261.0,22.540230,15.019581,0.0,20.0,22.0,28.0,100.0
how to recycle Christmas Trees,261.0,10.091954,20.865669,0.0,0.0,0.0,0.0,100.0
how to recycle Clothing,261.0,35.804598,21.991522,0.0,29.0,34.0,49.0,100.0
how to recycle Concrete,261.0,25.003831,20.131013,0.0,0.0,28.0,33.0,100.0
how to recycle Cooking Oil,261.0,25.118774,24.377742,0.0,0.0,32.0,38.0,100.0


In [18]:
# === Export pruned df to csv (for posterity!) === #
df3.to_csv("howtorecycle-pytrends.csv")

---

### `"Recycle" + {keyword}`

In [20]:
# === New blank dataframe === #
df_blank2 = pd.read_csv("blank_date_index.csv", index_col="date")
df_blank2.head()

""
date
2015-01-25
2015-02-01
2015-02-08
2015-02-15
2015-02-22


In [22]:
# === Run the function! === #
base_q2 = "recycle"

build_trend_df(base_q2, recycle_list, df_blank2)

Completed items 0-5.
Completed items 5-10.
Completed items 10-15.
Completed items 15-20.
Completed items 20-25.
Completed items 25-30.
No results for 30-35.
['recycle Blu-Ray Players', 'recycle Blue Glass Beverage Containers', 'recycle Blue Glass Containers', 'recycle Boomboxes', 'recycle Brake Fluid']
Completed items 30-35.
Completed items 35-40.
Completed items 40-45.
Completed items 45-50.
Completed items 50-55.
Completed items 55-60.
Completed items 60-65.
Completed items 65-70.
Completed items 70-75.
Completed items 75-80.
Completed items 80-85.
Completed items 85-90.
Completed items 90-95.
Completed items 95-100.
Completed items 100-105.
No results for 105-110.
['recycle Fungicides', 'recycle GPS Systems', 'recycle Game Consoles', 'recycle Garden Tools', 'recycle Gas Cylinders']
Completed items 105-110.
Completed items 110-115.
Completed items 115-120.
Completed items 120-125.
Completed items 125-130.
No results for 130-135.
['recycle Household Cleaners', 'recycle Household Furni

ResponseError: The request failed: Google returned a response with code 400.

In [23]:
# === Export
df_blank2

,recycle #1 Plastic,recycle #2 Plastic,recycle #3 Plastic,recycle #4 Plastic,recycle #5 Plastic,recycle #6 Plastic,recycle #7 Plastic,recycle Acids,recycle Adhesives,recycle Adult Toys,recycle Aerosol Cans - Full,recycle Air Conditioners,recycle Alkaline Batteries,recycle Aluminum Beverage Cans,recycle Aluminum Foil,recycle Aluminum Food Cans,recycle Aluminum Pie Plates,recycle Aluminum Trays,recycle Ammunition,recycle Antifreeze,recycle Arts and Crafts Supplies,recycle Asbestos,recycle Asphalt,recycle Audio Equipment,recycle Auto Bodies,recycle Auto Parts,recycle Baby Products,recycle Barbeque Grills,recycle Bicycles,recycle Bike Tires,recycle Branches,recycle Brick,recycle Brown Glass Beverage Containers,recycle Brown Glass Containers,recycle Brush,recycle Button Cell Batteries,recycle CD Cases,recycle CD Players,recycle CDs,recycle CFLs,recycle CRT Computer Monitors,recycle CRT Televisions,recycle Cables,recycle Calculators,recycle Candy Wrappers,recycle Car Batteries,recycle Car Fluids,recycle Carpet,recycle Carpet Padding,recycle Cassette Players,recycle Cassette Tapes,recycle Ceiling Tiles,recycle Cell Phone Accessories,recycle Cell Phones,recycle Ceramic Tile,recycle Chemistry Sets,recycle Chip Bags,recycle Christmas Trees,recycle Cigarettes,recycle Clear Glass Beverage Containers,recycle Clear Glass Containers,recycle Clothing,recycle Compostable Plastics,recycle Computer Peripherals - External,recycle Computer Peripherals - Internal,recycle Concrete,recycle Construction Debris,recycle Construction Materials,recycle Contaminated Soil,recycle Contaminated Wood,recycle Controlled Substances,recycle Cooking Oil,recycle Cookware,recycle Corks,recycle Corrugated Cardboard,recycle Crayons,recycle Curling Irons,recycle DVD Players,recycle DVDs,recycle Dehumidifiers,recycle Desktop Computers,recycle Digital Cameras,recycle Dirt,recycle Doors,recycle Drink Boxes,recycle Drink Pouches,recycle Electronic Servers,recycle Engine Degreasers,recycle Envelopes,recycle Exit Signs,recycle Explosives,recycle Eyeglasses,recycle Fabric,recycle Ferrous Metals,recycle Fertilizers,recycle Fire Extinguishers,recycle Flip-Flops,recycle Floppy Disks,recycle Fluorescent Tubes,recycle Freezers,...,recycle Musical Instruments - Non-metal,recycle Nail Polish,recycle Nail Polish Removers,recycle Neon Lights,recycle Newspaper,recycle Nickel-cadmium Batteries,recycle Nickel-metal Hydride Batteries,recycle Nickel-zinc Batteries,recycle Non-Organic Food Waste,recycle Nonferrous Metals,recycle Office Furniture,recycle Office Machines,recycle Office Paper,recycle Office Supplies,recycle Oil Filters,recycle Oil-Based Paint,recycle Pallets,recycle Paper Cups,recycle Paper Labels,recycle Paper Shredders,recycle Paper Sleeves,recycle Paperback Books,recycle Paperboard,recycle Perfumes,recycle Pesticide Containers,recycle Pesticides,recycle Phone Books,recycle Photographic Chemicals,recycle Pizza Boxes,recycle Plastic Bottle Accessories,recycle Plastic Buckets,recycle Plastic Cards,recycle Plastic Egg Cartons,recycle Plastic Furniture,recycle Plastic Packing Materials,recycle Plastic Paint Cans,recycle Power Tools,recycle Printers,recycle Projectors,recycle Propane Tanks,recycle Radiators,recycle Receivers,recycle Record Players,recycle Refrigerators,recycle Road Flares,recycle Roofing Materials,recycle Sand,recycle Sawdust,recycle Scanners,recycle Sealers,recycle Shingles,recycle Shoes,recycle Silver-oxide Batteries,recycle Six-pack Rings,recycle Small Appliances,recycle Smoke Detectors,recycle Solvents,recycle Sporting Goods,recycle Steel Cans,recycle Steel Lids,recycle Stone,recycle String Lights,recycle Stumps,recycle Surfboards,recycle Synthetic Cork,recycle Tablets,recycle Telephones,recycle Television Accessories,recycle Tennis Balls,recycle Tile,recycle Tires,recycle Toasters,recycle Toner Cartridges,recycle Tools,recycle Toothbrushes,recycle Toothpaste Tubes,recycle Toys,recycle Transmission Fluid,recycle Trophies,recycle Truck Tires,recycle Two-Way Ra

In [25]:
# === Export to csv for posterity === #
df_blank2.to_csv("recycle-pytrends_all.csv")

In [26]:
# === Drop 0-value columns === #

# Get list of 0-value columns
zero_value_cols = []

for col in df_blank2.columns:
    if df_blank2[col].mean() == 0.0:
        zero_value_cols.append(col)

df3 = df_blank2.drop(columns=zero_value_cols)
df3.shape

(261, 104)

In [29]:
# === Result is 78 columns - Check em out === #
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
df3.describe().T

,count,mean,std,min,25%,50%,75%,max
recycle #1 Plastic,261.0,12.754789,16.424923,0.0,0.0,0.0,29.0,67.0
recycle #2 Plastic,261.0,13.455939,17.856559,0.0,0.0,0.0,29.0,100.0
recycle #4 Plastic,261.0,15.743295,18.440111,0.0,0.0,0.0,30.0,69.0
recycle #5 Plastic,261.0,28.429119,19.684037,0.0,0.0,31.0,38.0,83.0
recycle #6 Plastic,261.0,22.425287,21.814409,0.0,0.0,31.0,37.0,100.0
recycle #7 Plastic,261.0,16.229885,20.611443,0.0,0.0,0.0,33.0,90.0
recycle Acids,261.0,15.413793,19.113760,0.0,0.0,0.0,33.0,78.0
recycle Aerosol Cans - Full,261.0,28.865900,14.368811,0.0,19.0,26.0,36.0,83.0
recycle Alkaline Batteries,261.0,36.681992,14.794048,0.0,26.0,34.0,43.0,100.0
recycle Aluminum Foil,261.0,33.011494,14.770418,0.0,21.0,32.0,41.0,90.0


In [28]:
# === Export pruned df to csv (for posterity!) === #
df3.to_csv("recycle-pytrends.csv")